In [20]:
import pandas as pd
from minio import Minio
from io import BytesIO

from minio import Minio
import pandas as pd
from io import BytesIO

# Conexión con MinIO en localhost
client = Minio(
    "localhost:9000",  # Endpoint de MinIO en localhost
    access_key="minioadmin",  # Tu clave de acceso
    secret_key="minioadmin",  # Tu clave secreta
    secure=False  # No usar HTTPS para MinIO local
)

# Verificar si el bucket existe
if not client.bucket_exists("access-zone"):
    print("Bucket 'access-zone' no encontrado.")
else:
    # Leer el archivo Parquet desde el bucket
    try:
        # Descargar el archivo Parquet
        obj = client.get_object("access-zone", "analytics/congestion_by_hour.parquet")
        
        # Leer el archivo Parquet en Pandas
        df = pd.read_parquet(BytesIO(obj.read()))
    except Exception as e:
        print(f"Error al leer el archivo Parquet: {e}")




In [31]:
high_congestion_df = df[df['congestion_level'] == 'high']

# Mostrar las horas con mayor congestión y los vehículos predominantes
high_congestion_summary = high_congestion_df[['hour', 'congestion_level', 'vehiculo_predominante']]

high_congestion_summary



,hour,congestion_level,vehiculo_predominante
0,0,high,cars
13,6,high,cars
18,8,high,cars
21,9,high,cars
26,11,high,cars
30,12,high,cars
36,14,high,cars
39,15,high,cars
42,16,high,cars
48,18,high,cars


In [ ]:
# contar las ocurrencias de los tipos de vehículos predominantes en las horas de alta congestión:
vehicle_count = high_congestion_df['vehiculo_predominante'].value_counts()

print("\nTipos de vehículos predominantes en horarios de alta congestión:")
print(vehicle_count)




Tipos de vehículos predominantes en horarios de alta congestión:


vehiculo_predominante
cars    14
Name: count, dtype: int64

In [29]:
# Análisis de las franjas horarias con mayor congestión (se puede hacer por cantidad de vehículos, por ejemplo)
high_congestion_by_hour = high_congestion_df.groupby('hour').agg({'total_vehicles': 'sum'}).sort_values('total_vehicles', ascending=False)

print("\nFranjas horarias con mayor congestión total de vehículos:")
high_congestion_by_hour


Franjas horarias con mayor congestión total de vehículos:


,total_vehicles
hour,
8,9926
18,9330
9,8830
19,5688
12,4835
16,4622
11,4376
21,4144
22,3844
